# Truy vấn và Phân tích Dữ liệu từ TimescaleDB

Notebook này minh họa cách kết nối tới TimescaleDB, truy vấn dữ liệu chất lượng không khí và trực quan hóa kết quả.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
# TimescaleDB connection settings
DB_NAME = "airquality_timeseries"  # Change this to your database name
DB_USER = "your_username"          # Change this to your database username
DB_PASSWORD = "your_password"      # Change this to your database password
DB_HOST = "your-timescale-host"    # Change this to your TimescaleDB host
DB_PORT = "5432"                   # Change this port if different

# Create connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine for pandas
engine = create_engine(connection_string)

# Test connection
try:
    # Connect using psycopg2 to check if TimescaleDB extension is available
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    cursor = conn.cursor()
    
    # Check if TimescaleDB extension is enabled
    cursor.execute("SELECT extname FROM pg_extension WHERE extname = 'timescaledb'")
    result = cursor.fetchone()
    
    if result:
        print("✅ Successfully connected to TimescaleDB!")
    else:
        print("⚠️ Connected to PostgreSQL but TimescaleDB extension is not enabled.")
        
    # Close psycopg2 connection
    cursor.close()
    conn.close()
    
except Exception as e:
    print(f"❌ Failed to connect: {e}")

## Truy vấn Dữ liệu Cơ bản

In [ ]:
# Query to get a sample of the data
query = """
SELECT * FROM air_quality_measurements
ORDER BY time DESC
LIMIT 10;
"""

# Load data into a pandas DataFrame
df_sample = pd.read_sql(query, engine)
df_sample

## Tổng quan Dữ liệu

In [ ]:
# Get summary statistics for different stations
station_summary_query = """
SELECT 
    station,
    COUNT(*) as record_count,
    MIN(time) as first_record,
    MAX(time) as last_record,
    COUNT(DISTINCT date_trunc('day', time)) as days_count
FROM 
    air_quality_measurements
GROUP BY 
    station
ORDER BY 
    record_count DESC;
"""

df_summary = pd.read_sql(station_summary_query, engine)
df_summary

## Phân tích Theo Thời gian (Time Series Analysis)

In [ ]:
# Get time series data for PM2.5 for the last 7 days for all stations
time_series_query = """
SELECT 
    time, 
    station, 
    data_type,
    "PM2.5"
FROM 
    air_quality_measurements
WHERE 
    time > NOW() - INTERVAL '7 days' AND
    "PM2.5" IS NOT NULL
ORDER BY 
    time;
"""

df_time_series = pd.read_sql(time_series_query, engine)

# Plot time series for each station
plt.figure(figsize=(14, 8))

# Plot for data_type='stat'
stat_data = df_time_series[df_time_series['data_type'] == 'stat']
for station in stat_data['station'].unique():
    station_data = stat_data[stat_data['station'] == station]
    plt.plot(station_data['time'], station_data['PM2.5'], label=f"{station} (stat)")

# Thêm labels và titles
plt.title('PM2.5 Concentration over Last 7 Days')
plt.xlabel('Time')
plt.ylabel('PM2.5 Concentration (µg/m³)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## So sánh Nồng độ Trung bình Giữa các Trạm Đo

In [ ]:
# Get average PM2.5, PM10, and NO2 for each station over the last month
avg_query = """
SELECT 
    station,
    data_type,
    AVG("PM2.5") as avg_pm25,
    AVG("PM10") as avg_pm10,
    AVG("NO2") as avg_no2,
    AVG("SO2") as avg_so2,
    AVG("CO") as avg_co,
    AVG("O3") as avg_o3
FROM 
    air_quality_measurements
WHERE 
    time > NOW() - INTERVAL '30 days'
GROUP BY 
    station, data_type
ORDER BY 
    station, data_type;
"""

df_avg = pd.read_sql(avg_query, engine)

# Filter to only show 'stat' type
df_avg_stat = df_avg[df_avg['data_type'] == 'stat']

# Set up the figure
plt.figure(figsize=(16, 10))

# Create a bar chart for average PM2.5
plt.subplot(2, 2, 1)
sns.barplot(x='station', y='avg_pm25', data=df_avg_stat)
plt.title('Average PM2.5 by Station')
plt.xticks(rotation=45)
plt.ylabel('PM2.5 (µg/m³)')

# Create a bar chart for average PM10
plt.subplot(2, 2, 2)
sns.barplot(x='station', y='avg_pm10', data=df_avg_stat)
plt.title('Average PM10 by Station')
plt.xticks(rotation=45)
plt.ylabel('PM10 (µg/m³)')

# Create a bar chart for average NO2
plt.subplot(2, 2, 3)
sns.barplot(x='station', y='avg_no2', data=df_avg_stat)
plt.title('Average NO2 by Station')
plt.xticks(rotation=45)
plt.ylabel('NO2 (µg/m³)')

# Create a bar chart for average SO2
plt.subplot(2, 2, 4)
sns.barplot(x='station', y='avg_so2', data=df_avg_stat)
plt.title('Average SO2 by Station')
plt.xticks(rotation=45)
plt.ylabel('SO2 (µg/m³)')

plt.tight_layout()
plt.show()

## Phân tích Mẫu Theo Thời gian trong Ngày (Daily Patterns)

In [ ]:
# Analyze hourly patterns in PM2.5 levels
hourly_pattern_query = """
SELECT 
    EXTRACT(HOUR FROM time) as hour_of_day,
    station,
    AVG("PM2.5") as avg_pm25
FROM 
    air_quality_measurements
WHERE 
    data_type = 'stat' AND
    "PM2.5" IS NOT NULL AND
    time > NOW() - INTERVAL '30 days'
GROUP BY 
    hour_of_day, station
ORDER BY 
    hour_of_day, station;
"""

df_hourly = pd.read_sql(hourly_pattern_query, engine)

# Plot hourly patterns for each station
plt.figure(figsize=(14, 8))

for station in df_hourly['station'].unique():
    station_data = df_hourly[df_hourly['station'] == station]
    plt.plot(station_data['hour_of_day'], station_data['avg_pm25'], marker='o', linestyle='-', label=station)

plt.title('Average PM2.5 by Hour of Day (Last 30 Days)')
plt.xlabel('Hour of Day (24-hour)')
plt.ylabel('Average PM2.5 (µg/m³)')
plt.xticks(range(0, 24))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Phân tích Tương quan giữa các Chất Ô nhiễm

In [ ]:
# Query for correlation analysis
correlation_query = """
SELECT 
    "PM2.5", "PM10", "NO2", "SO2", "CO", "O3"
FROM 
    air_quality_measurements
WHERE 
    data_type = 'stat' AND
    time > NOW() - INTERVAL '30 days';
"""

df_corr = pd.read_sql(correlation_query, engine)

# Calculate correlation matrix
correlation_matrix = df_corr.corr()

# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation between Air Pollutants')
plt.tight_layout()
plt.show()

## Truy vấn Cận Ngày Thời gian (Time Bucket)

In [ ]:
# Use time_bucket (TimescaleDB specific function) to get daily averages
time_bucket_query = """
SELECT 
    time_bucket('1 day', time) as day,
    station,
    AVG("PM2.5") as avg_pm25,
    MAX("PM2.5") as max_pm25,
    MIN("PM2.5") as min_pm25
FROM 
    air_quality_measurements
WHERE 
    data_type = 'stat' AND
    time > NOW() - INTERVAL '30 days'
GROUP BY 
    day, station
ORDER BY 
    day, station;
"""

df_daily = pd.read_sql(time_bucket_query, engine)

# Plot daily averages for each station
plt.figure(figsize=(14, 8))

for station in df_daily['station'].unique():
    station_data = df_daily[df_daily['station'] == station]
    plt.plot(station_data['day'], station_data['avg_pm25'], marker='o', linestyle='-', label=f"{station} (avg)")
    plt.fill_between(station_data['day'], 
                     station_data['min_pm25'], 
                     station_data['max_pm25'], 
                     alpha=0.2)

plt.title('Daily Average PM2.5 with Min-Max Range (Last 30 Days)')
plt.xlabel('Date')
plt.ylabel('PM2.5 (µg/m³)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()